In [58]:
import pandas as pd
import numpy as np

In [ ]:
# Read in Datasets
games = pd.read_csv(r'..\data\raw\Games.csv', dtype={"gameSubLabel": str})
team_stats = pd.read_csv(r'..\data\raw\TeamStatistics.csv')

In [ ]:
# Merges games onto team_stats using gameId
team_game = team_stats.merge(
    games[['gameId', 'homeScore', 'awayScore', 'hometeamId', 'awayteamId', 'attendance', 'arenaId']],
    on='gameId',
    how='left'
)

In [ ]:
#
team_game['opponentTeamId'] = np.where(
    team_game['teamId'] == team_game['hometeamId'],
    team_game['awayteamId'],
    team_game['hometeamId']
)

team_game['opponentScore'] = np.where(
    team_game['teamId'] == team_game['hometeamId'],
    team_game['awayScore'],
    team_game['homeScore']
)

In [80]:
team_game['is_home'] = team_game['home'].astype(bool)

In [ ]:
# Calculating eFG% (Effective Field Goal Percentage)
team_game['efg'] = (team_game['fieldGoalsMade'] + 0.5 * team_game['threePointersMade']) / team_game['fieldGoalsAttempted']
team_game['efg'] = team_game['efg'].fillna(0)

In [ ]:
# Calculating True Shooting % 
team_game['ts'] = team_game['teamScore'] / (2 * (team_game['fieldGoalsAttempted'] + 0.44 * team_game['freeThrowsAttempted']))
team_game['ts'] = team_game['ts'].fillna(0)

In [ ]:
# Calculating Possessions
team_game['possessions'] = team_game['fieldGoalsAttempted'] - team_game['reboundsOffensive'] + team_game['turnovers'] + (0.4 * team_game['freeThrowsAttempted'])

# Replace any 0's with NaN to avoid divide by zer0 error later on
team_game['possessions'] = team_game['possessions'].replace(0, np.nan)

In [ ]:
# Offensive Rating (Point per 100 Possessions)
team_game['off_rating'] = team_game['teamScore'] / team_game['possessions'] * 100

# Defensive Rating (Opp Points per 100 Possessions)
team_game['def_rating'] = team_game['opponentScore'] / team_game['possessions'] * 100

# Turnovers per Possession
team_game['to_per_poss'] = team_game['turnovers'] / team_game['possessions']